In [28]:
using CSV
using DataFrames
using .IAI
using Random
using Statistics

# ---------------------------------------------------------
# Helper: get ENV with default
# ---------------------------------------------------------
getenv(key::AbstractString, default::AbstractString) =
    haskey(ENV, key) ? ENV[key] : default

# ---------------------------------------------------------
# 1. Resolve config from environment
# ---------------------------------------------------------

data_file   = getenv("TRAIN_DATA_FILE", "../data_gen/train_countdown_results_with_prompt_augmented_xgb.csv")
reward_file = getenv("REWARD_MATRIX_FILE", "../data_gen/reward_matrix_xgb.csv")

seed = parse(Int, getenv("RANDOM_STATE", "42"))

println("===================================================================")
println("OPTIMAL POLICY TREE (PROMPT POLICY VIA XGBOOST REWARD MATRIX)")
println("===================================================================")
println("Instance data file : ", data_file)
println("Reward matrix file : ", reward_file)
println("Random seed        : ", seed)
println("===================================================================")

# ---------------------------------------------------------
# 2. Load instance data and reward matrix
# ---------------------------------------------------------

df_instances = CSV.read(data_file, DataFrame)
println("Instance rows (raw): ", nrow(df_instances))
println("Columns in instance data:")
println(names(df_instances))

reward_df = CSV.read(reward_file, DataFrame)
println("\nReward matrix shape: ", size(reward_df))
println("Columns in reward matrix:")
println(names(reward_df))

# ---------------------------------------------------------
# 2b. Rename reward columns (template codes) to interpretable labels
#     - Encode ALL binary features + conciseness + few-shot + length
#     - Do NOT add '0shot' when few_shot == 0
# ---------------------------------------------------------

function pretty_template_label(col::AbstractString)::String
    parts = split(col, '_')
    # Template codes from build_reward_matrix: 8 binaries + few_shot + len_bin
    if length(parts) != 10
        return col
    end

    paraphr        = parts[1]  # 0/1
    role_spec      = parts[2]  # 0/1
    reasoning_trig = parts[3]  # 0/1
    chain_thought  = parts[4]  # 0/1
    self_check     = parts[5]  # 0/1
    conc           = parts[6]  # 0/1
    verb           = parts[7]  # 0/1
    context_exp    = parts[8]  # 0/1
    few_shot       = parts[9]  # "0","1","2","3",...
    len_bin        = parts[10] # "S" or "L"

    feats = String[]

    if paraphr == "1"
        push!(feats, "paraphrasing")
    end
    if role_spec == "1"
        push!(feats, "role")
    end
    if reasoning_trig == "1"
        push!(feats, "reasoning")
    end
    if chain_thought == "1"
        push!(feats, "cot")
    end
    if self_check == "1"
        push!(feats, "selfcheck")
    end
    if conc == "1"
        push!(feats, "concise")
    end
    if verb == "1"
        push!(feats, "verbose")
    end
    if context_exp == "1"
        push!(feats, "context")
    end

    # Only add few-shot if > 0
    if few_shot != "0"
        push!(feats, few_shot * "shot")   # "1shot", "2shot", ...
    end

    # Always append length bin
    push!(feats, len_bin)

    if isempty(feats)
        return "base_" * len_bin
    else
        return join(feats, "_")
    end
end

for col in names(reward_df)
    col_str = String(col)
    if col_str == "instance_id"
        continue
    end
    new_name = pretty_template_label(col_str)
    if new_name != col_str
        rename!(reward_df, col => new_name)  # strings only
    end
end

println("\nReward matrix columns AFTER renaming:")
println(names(reward_df))

# ---------------------------------------------------------
# 2a. Robustly ensure "instance_id" exists in both tables
# ---------------------------------------------------------

function ensure_instance_id!(df::DataFrame; context::String = "")
    cols = names(df)
    if "instance_id" in cols
        return
    end
    # Try to find a column whose string name is "instance_id"
    str_cols = String.(cols)
    if "instance_id" in str_cols
        old = cols[findfirst(==("instance_id"), str_cols)]
        rename!(df, old => "instance_id")
        return
    end
    # Fallback: if there is an :id column, treat it as instance_id
    if :id in cols
        rename!(df, :id => "instance_id")
        return
    end
    error("Could not find instance_id in $(context) data. Available columns: $(cols)")
end

ensure_instance_id!(df_instances, context = "instance")
ensure_instance_id!(reward_df,    context = "reward")

@assert "instance_id" in names(df_instances) "instance_id must be present in instance data."
@assert "instance_id" in names(reward_df)    "instance_id must be present in reward matrix."

# ---------------------------------------------------------
# 3. Build unique instance feature rows (one row per instance_id)
# ---------------------------------------------------------

all_inst_cols = [
    c for c in names(df_instances) if startswith(String(c), "inst_")
]

drop_inst = [
    "inst_expr_depth",
    "inst_count_add",
    "inst_count_sub",
    "inst_count_mul",
    "inst_count_div",
    "inst_noncomm_ops",
]

# *** IMPORTANT: instance_id is NOT in inst_cols ***
inst_cols = [c for c in all_inst_cols if !(c in drop_inst)]

@assert !isempty(inst_cols) "No instance feature columns found starting with 'inst_'."

println("\nUsing instance feature columns (", length(inst_cols), "):")
println(inst_cols)

# groupby keeps the group key ("instance_id") automatically
agg = combine(groupby(df_instances, "instance_id")) do sdf
    sdf[1, inst_cols]
end

# agg has: "instance_id" + inst_* columns
println("\nUnique instances after aggregation: ", nrow(agg))
println("Columns in aggregated instance table:")
println(names(agg))

# ---------------------------------------------------------
# 4. Join instance features with reward matrix
# ---------------------------------------------------------

joined = innerjoin(agg, reward_df, on = "instance_id")
println("Rows after joining with reward matrix: ", nrow(joined))

# X: ONLY instance features (no instance_id)
X = select(joined, inst_cols)

# reward columns = everything else that's not instance_id or inst_*
reward_cols = setdiff(names(joined), vcat(["instance_id"], inst_cols))
rewards = select(joined, reward_cols)

println("Number of templates (reward columns): ", length(reward_cols))

# ---------------------------------------------------------
# 5. Train/test split (by instance)
# ---------------------------------------------------------

Random.seed!(seed)

n = nrow(X)
idx = Random.shuffle(1:n)
cut = Int(floor(0.6 * n))

train_idx = idx[1:cut]
test_idx  = idx[cut+1:end]

train_X       = X[train_idx, :]
test_X        = X[test_idx, :]
train_rewards = rewards[train_idx, :]
test_rewards  = rewards[test_idx, :]

train_instance_ids = joined.instance_id[train_idx]
test_instance_ids  = joined.instance_id[test_idx]

println("\nTrain size (instances): ", nrow(train_X))
println("Test size  (instances): ", nrow(test_X))

# ---------------------------------------------------------
# 6. Fit Optimal Policy Tree on reward matrix
# ---------------------------------------------------------

policy = IAI.GridSearch(
    IAI.OptimalTreePolicyMaximizer(
        random_seed = seed,
    ),
    max_depth   = 3,
    minbucket   = 5
)

println("\nFitting OptimalTreePolicyMaximizer on reward matrix...")
IAI.fit!(policy, train_X, train_rewards)

println("\n=================== Learned Optimal Policy Tree ===================")
display(policy)
println("===================================================================")

# ---------------------------------------------------------
# 7. Predict best template and its expected reward on test set
# ---------------------------------------------------------

policy_actions  = IAI.predict(policy, test_X)
policy_outcomes = IAI.predict_outcomes(policy, test_X, test_rewards)

println("\nNumber of unique prescribed templates on test: ",
        length(unique(policy_actions)))

mean_policy_reward = mean(policy_outcomes)
println("Mean XGBoost-predicted reward under learned policy (test): ",
        mean_policy_reward)

OPTIMAL POLICY TREE (PROMPT POLICY VIA XGBOOST REWARD MATRIX)
Instance data file : ../data_gen/train_countdown_results_with_prompt_augmented_xgb.csv
Reward matrix file : ../data_gen/reward_matrix_xgb.csv
Random seed        : 42
Instance rows (raw): 43445
Columns in instance data:
["prompt_id", "instance_id", "prompt", "correct", "message", "inst_n_numbers", "inst_range", "inst_std", "inst_count_small", "inst_count_large", "inst_count_duplicates", "inst_count_even", "inst_count_odd", "inst_count_div_2", "inst_count_div_3", "inst_count_div_5", "inst_count_div_7", "inst_count_primes", "inst_distance_simple", "inst_distance_max", "inst_distance_avg", "inst_easy_pairs", "inst_log_target", "inst_expr_depth", "inst_count_add", "inst_count_sub", "inst_count_mul", "inst_count_div", "inst_noncomm_ops", "numbers", "target", "solution", "prompt_paraphrasing", "prompt_role-specification", "prompt_reasoning-trigger", "prompt_chain-of-thought", "prompt_self-check", "prompt_conciseness", "prompt_verbo

All Grid Results:

 Row │ minbucket  max_depth  cp          train_score  valid_score  rank_valid_score
     │ Int64      Int64      Float64     Float64      Float64      Int64
─────┼──────────────────────────────────────────────────────────────────────────────
   1 │         5          3  0.00232372     0.144213     0.120366                 1

Best Params:
  minbucket => 5
  cp => 0.0023237249630371215
  max_depth => 3

Best Model - Fitted OptimalTreePolicyMaximizer:
  1) Split: inst_n_numbers < 3.5
    2) Prescribe: concise_3shot_S, 591 points, error 0.01589
    3) Split: inst_range < 64.5
      4) Prescribe: paraphrasing_selfcheck_verbose_1shot_L, 338 points, error 0.3137
      5) Split: inst_count_div_3 < 1.5
        6) Prescribe: concise_3shot_S, 146 points, error 0.399
        7) Prescribe: selfcheck_concise_S, 125 points, error 0.6018

In [29]:
# ---------------------------------------------------------
# 8. Baseline: best single template overall (train) vs policy
# ---------------------------------------------------------

template_means = Dict{String, Float64}()

for col_name in names(train_rewards)
    col_str = String(col_name)
    template_means[col_str] = mean(train_rewards[!, col_name])
end

best_single_reward, best_single_template_name = findmax(template_means)

println("\nBest single template on TRAIN:")
println("  Template: ", best_single_template_name)
println("  Mean reward (train): ", best_single_reward)

single_policy_reward = mean(test_rewards[!, best_single_template_name])

println("Mean reward using best single template on TEST: ",
        single_policy_reward)
println("Mean reward using learned policy on TEST:      ",
        mean_policy_reward)

# ---------------------------------------------------------
# 9. Save results
# ---------------------------------------------------------

output = hcat(
    DataFrame(instance_id   = test_instance_ids),
    test_X,
    DataFrame(
        assigned_template = policy_actions,
        predicted_reward  = policy_outcomes,
    ),
)

CSV.write("opt_policy_xgb_results.csv", output)
println("\nSaved results to opt_policy_xgb_results.csv")

println("\nDone.")


Best single template on TRAIN:
  Template: paraphrasing_verbose_1shot_S
  Mean reward (train): 0.7563809637508333
Mean reward using best single template on TEST: 0.7526994510824997
Mean reward using learned policy on TEST:      0.7798626929649997

Saved results to opt_policy_xgb_results.csv

Done.


In [ ]:
# Tentative Real Test Set

using CSV
using DataFrames
using .IAI
using Random
using Statistics

# ---------------------------------------------------------
# Helper: get ENV with default
# ---------------------------------------------------------
getenv(key::AbstractString, default::AbstractString) =
    haskey(ENV, key) ? ENV[key] : default

# ---------------------------------------------------------
# 1. Resolve config from environment
# ---------------------------------------------------------

train_data_file = getenv("TRAIN_DATA_FILE",
    "../data_gen/train_countdown_results_with_prompt_augmented_xgb.csv")
test_data_file  = getenv("TEST_DATA_FILE",
    "../data_gen/test_countdown_results_with_prompt_augmented_xgb.csv")

# Reward matrices: allow TRAIN_REWARD_MATRIX_FILE or fallback to REWARD_MATRIX_FILE
train_reward_file = getenv(
    "TRAIN_REWARD_MATRIX_FILE",
    getenv("REWARD_MATRIX_FILE", "../data_gen/reward_matrix_xgb.csv"),
)
test_reward_file = getenv(
    "TEST_REWARD_MATRIX_FILE",
    train_reward_file,
)

seed = parse(Int, getenv("RANDOM_STATE", "42"))

println("===================================================================")
println("OPTIMAL POLICY TREE (PROMPT POLICY VIA XGBOOST REWARD MATRIX)")
println("===================================================================")
println("Train instance file : ", train_data_file)
println("Test  instance file : ", test_data_file)
println("Train reward file   : ", train_reward_file)
println("Test  reward file   : ", test_reward_file)
println("Random seed         : ", seed)
println("===================================================================")

# ---------------------------------------------------------
# 2. Load train/test instance data and reward matrices
# ---------------------------------------------------------

println("Loading training instance data...")
train_instances = CSV.read(train_data_file, DataFrame)
println("Train instance rows (raw): ", nrow(train_instances))
println("Train instance columns:")
println(names(train_instances))

println("\nLoading test instance data...")
test_instances = CSV.read(test_data_file, DataFrame)
println("Test instance rows (raw): ", nrow(test_instances))
println("Test instance columns:")
println(names(test_instances))

println("\nLoading training reward matrix...")
reward_train_df = CSV.read(train_reward_file, DataFrame)
println("Train reward matrix shape: ", size(reward_train_df))
println("Train reward columns (raw):")
println(names(reward_train_df))

println("\nLoading test reward matrix...")
reward_test_df = CSV.read(test_reward_file, DataFrame)
println("Test reward matrix shape: ", size(reward_test_df))
println("Test reward columns (raw):")
println(names(reward_test_df))

# ---------------------------------------------------------
# 2a. Ensure "instance_id" exists in all tables
# ---------------------------------------------------------

function ensure_instance_id!(df::DataFrame; context::String = "")
    cols = names(df)
    if "instance_id" in cols
        return
    end
    str_cols = String.(cols)
    if "instance_id" in str_cols
        old = cols[findfirst(==("instance_id"), str_cols)]
        rename!(df, old => "instance_id")
        return
    end
    if "id" in str_cols
        old = cols[findfirst(==("id"), str_cols)]
        rename!(df, old => "instance_id")
        return
    end
    error("Could not find instance_id in $(context) data. Available columns: $(cols)")
end

ensure_instance_id!(train_instances, context = "train instance")
ensure_instance_id!(test_instances,  context = "test instance")
ensure_instance_id!(reward_train_df, context = "train reward")
ensure_instance_id!(reward_test_df,  context = "test reward")

@assert "instance_id" in names(train_instances)
@assert "instance_id" in names(test_instances)
@assert "instance_id" in names(reward_train_df)
@assert "instance_id" in names(reward_test_df)

# ---------------------------------------------------------
# 2b. Human-readable reward column names (injective)
# ---------------------------------------------------------

function pretty_template_name(code::String)
    parts = split(code, "_")
    if length(parts) != 10
        return code
    end

    p  = parse(Int, parts[1])  # paraphrasing
    r  = parse(Int, parts[2])  # role
    rt = parse(Int, parts[3])  # reasoning-trigger
    ct = parse(Int, parts[4])  # chain-of-thought
    sc = parse(Int, parts[5])  # self-check
    cc = parse(Int, parts[6])  # conciseness
    vb = parse(Int, parts[7])  # verbosity
    cx = parse(Int, parts[8])  # context-expansion
    shots = parse(Int, parts[9])
    lenflag = parts[10] == "L" ? "L" : "S"

    tags = String[]

    # base "families"
    if p == 1
        push!(tags, "paraphrasing")
    end
    if r == 1
        push!(tags, "role")
    end
    if rt == 1
        push!(tags, "reasoning")
    end
    if ct == 1
        push!(tags, "cot")
    end
    if sc == 1
        push!(tags, "selfcheck")
    end

    # style modifiers
    if cc == 1
        push!(tags, "concise")
    end
    if vb == 1
        push!(tags, "verbose")
    end
    if cx == 1
        push!(tags, "context")
    end

    if isempty(tags)
        push!(tags, "base")
    end

    name = join(tags, "_")

    if shots > 0
        name *= "_" * string(shots, "shot")
    end

    name *= "_" * lenflag

    # attach original code to guarantee uniqueness
    return name * "__" * code
end

function rename_reward_columns!(df::DataFrame)
    mapping = Dict{String,String}()
    for cname in names(df)
        if cname == "instance_id"
            continue
        end
        mapping[cname] = pretty_template_name(cname)
    end
    rename!(df, mapping)
end

println("\nRenaming reward columns to human-readable labels (train)...")
rename_reward_columns!(reward_train_df)
println("Train reward columns AFTER renaming:")
println(names(reward_train_df))

println("\nRenaming reward columns to human-readable labels (test)...")
rename_reward_columns!(reward_test_df)
println("Test reward columns AFTER renaming:")
println(names(reward_test_df))

# ---------------------------------------------------------
# 3. Build instance-level feature rows (one row per instance_id)
# ---------------------------------------------------------

all_inst_cols = [
    c for c in names(train_instances) if startswith(String(c), "inst_")
]

drop_inst = [
    "inst_expr_depth",
    "inst_count_add",
    "inst_count_sub",
    "inst_count_mul",
    "inst_count_div",
    "inst_noncomm_ops",
]

inst_cols = [c for c in all_inst_cols if !(c in drop_inst)]

@assert !isempty(inst_cols) "No instance feature columns found starting with 'inst_'."

println("\nUsing instance feature columns (", length(inst_cols), "):")
println(inst_cols)

train_agg = combine(groupby(train_instances, "instance_id")) do sdf
    sdf[1, inst_cols]
end

test_agg = combine(groupby(test_instances, "instance_id")) do sdf
    sdf[1, inst_cols]
end

println("\nUnique train instances after aggregation: ", nrow(train_agg))
println("Columns in aggregated TRAIN instance table:")
println(names(train_agg))

println("\nUnique test instances after aggregation: ", nrow(test_agg))
println("Columns in aggregated TEST instance table:")
println(names(test_agg))

# ---------------------------------------------------------
# 4. Join instance features with reward matrices (train & test)
# ---------------------------------------------------------

train_joined = innerjoin(train_agg, reward_train_df, on = "instance_id")
println("\nRows after joining TRAIN instances with TRAIN reward matrix: ",
        nrow(train_joined))

test_joined = innerjoin(test_agg, reward_test_df, on = "instance_id")
println("Rows after joining TEST instances with TEST reward matrix: ",
        nrow(test_joined))

# X: ONLY instance features (no instance_id)
X_train = select(train_joined, inst_cols)
X_test  = select(test_joined, inst_cols)

# reward columns = everything else that's not instance_id or inst_*
train_reward_cols = setdiff(names(train_joined), vcat(["instance_id"], inst_cols))
test_reward_cols  = setdiff(names(test_joined),  vcat(["instance_id"], inst_cols))

reward_cols = intersect(train_reward_cols, test_reward_cols)
@assert !isempty(reward_cols) "No common reward columns between train and test."

println("\nNumber of templates (reward columns): ", length(reward_cols))

train_rewards = select(train_joined, reward_cols)
test_rewards  = select(test_joined,  reward_cols)

train_instance_ids = train_joined[!, "instance_id"]
test_instance_ids  = test_joined[!, "instance_id"]

println("\nTrain size (instances): ", nrow(X_train))
println("Test size  (instances): ", nrow(X_test))

# ---------------------------------------------------------
# 5. Fit Optimal Policy Tree on FULL TRAINING SET
# ---------------------------------------------------------

Random.seed!(seed)

policy = IAI.OptimalTreePolicyMaximizer(
    random_seed = seed,
    max_depth   = 3,
    minbucket   = 5,
    cp          = 0.0,
)

println("\nFitting OptimalTreePolicyMaximizer on FULL TRAIN reward matrix...")
IAI.fit!(policy, X_train, train_rewards)

println("\n=================== Learned Optimal Policy Tree ===================")
display(policy)
println("===================================================================")

# ---------------------------------------------------------
# 6. Predict best template and expected reward: TRAIN & TEST
# ---------------------------------------------------------

policy_actions_train  = IAI.predict(policy, X_train)
policy_outcomes_train = IAI.predict_outcomes(policy, X_train, train_rewards)

policy_actions_test  = IAI.predict(policy, X_test)
policy_outcomes_test = IAI.predict_outcomes(policy, X_test, test_rewards)

println("\nNumber of unique prescribed templates on TRAIN: ",
        length(unique(policy_actions_train)))
println("Number of unique prescribed templates on TEST:  ",
        length(unique(policy_actions_test)))

mean_policy_reward_train = mean(policy_outcomes_train)
mean_policy_reward_test  = mean(policy_outcomes_test)

println("\nMean XGBoost-predicted reward under learned policy (TRAIN): ",
        mean_policy_reward_train)
println("Mean XGBoost-predicted reward under learned policy (TEST):  ",
        mean_policy_reward_test)

# ---------------------------------------------------------
# 7. Baseline: best single template overall (TRAIN) vs policy
# ---------------------------------------------------------

template_means = Dict{String, Float64}()

for col_name in reward_cols
    template_means[col_name] = mean(train_rewards[!, col_name])
end

best_single_template_name = ""
best_single_reward = -Inf

for (name, val) in template_means
    if val > best_single_reward
        best_single_reward = val
        best_single_template_name = name
    end
end

println("\nBest single template on TRAIN:")
println("  Template: ", best_single_template_name)
println("  Mean reward (TRAIN): ", best_single_reward)

single_policy_reward_train = mean(train_rewards[!, best_single_template_name])
single_policy_reward_test  = mean(test_rewards[!,  best_single_template_name])

println("\nMean reward using best single template on TRAIN: ",
        single_policy_reward_train)
println("Mean reward using best single template on TEST:  ",
        single_policy_reward_test)
println("Mean reward using learned policy on TEST:        ",
        mean_policy_reward_test)

# ---------------------------------------------------------
# 8. Save TEST results
# ---------------------------------------------------------

output_test = hcat(
    DataFrame(instance_id   = test_instance_ids),
    X_test,
    DataFrame(
        assigned_template = policy_actions_test,
        predicted_reward  = policy_outcomes_test,
    ),
)

CSV.write("opt_policy_xgb_test_results.csv", output_test)
println("\nSaved TEST results to opt_policy_xgb_test_results.csv")

println("\nDone.")